In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")

utterance = "I am facing issues with slack. Facing background audio echo."
doc = nlp(utterance)

for token in doc:
    print(token.text, token.dep_, token.head.text, token.pos_, token.lemma_)


I nsubj facing PRON I
am aux facing AUX be
facing ROOT facing VERB face
issues dobj facing NOUN issue
with prep issues ADP with
slack pobj with NOUN slack
. punct facing PUNCT .
Facing ROOT Facing VERB face
background compound echo NOUN background
audio compound echo NOUN audio
echo dobj Facing NOUN echo
. punct Facing PUNCT .


In [11]:
# !python -m spacy download en_core_web_sm


In [13]:
core_concepts = []

for token in doc:
    if token.pos_ in ("VERB", "NOUN", "PROPN") and token.dep_ in ("ROOT", "dobj", "pobj", "nsubj", "attr"):
        concept = token.lemma_.lower()
        core_concepts.append(concept)


In [14]:
core_concepts

['face', 'issue', 'slack', 'face', 'echo']

In [15]:
import networkx as nx
G = nx.DiGraph()

# Add directed edges: head → dependent
for token in doc:
    if token.pos_ in ("VERB", "NOUN", "PROPN"):
        head = token.head.lemma_.lower()
        child = token.lemma_.lower()
        if head != child:
            G.add_edge(head, child, label=token.dep_)


In [16]:
# Find root verb
root = [token.lemma_.lower() for token in doc if token.dep_ == "ROOT"]
paths = []

for r in root:
    for target in G.successors(r):
        path = [r, target]
        for next_node in G.successors(target):
            path.append(next_node)
        paths.append(" ".join(path))


In [17]:
paths

['face issue',
 'face echo background audio',
 'face issue',
 'face echo background audio']

In [22]:
import spacy


def extract_important_words(text):
    doc = nlp(text)

    important_words = []

    for token in doc:
        if not token.is_stop and not token.is_punct:
            if token.pos_ in ("NOUN", "PROPN", "VERB", "ADJ"):
                important_words.append(token.lemma_.lower())

    return important_words

# Example
text = "I am facing issues with slack. How to remove background echo"
print(extract_important_words(text))


['team', 'need', 'laptop', 'check', 'key', 'work', 'need', 'file', 'ticket']


In [24]:
import spacy
nlp = spacy.load("en_core_web_sm")

def extract_important_words(text):
    doc = nlp(text)
    important_words = []

    for token in doc:
        lemma = token.lemma_.lower()
        if token.is_stop or token.is_punct:
            continue

        # Keep informative nouns, adjectives, and main verbs
        if token.pos_ in ("NOUN", "PROPN", "ADJ"):
            important_words.append(lemma)
        elif token.pos_ == "VERB" and token.dep_ in ("ROOT", "xcomp", "acl", "advcl"):
            important_words.append(lemma)

    return important_words

# Example
text = "I am facing issues with slack. How to remove background echo"
print(extract_important_words(text))


['face', 'issue', 'slack', 'remove', 'background', 'echo']


In [25]:
!pip install RapidFuzz

     |████████████████████████████████| 1.5 MB 614 kB/s eta 0:00:01
You should consider upgrading via the '/Users/abhishekbairagi/Desktop/exp/envs/base/bin/python3 -m pip install --upgrade pip' command.


In [31]:
from rapidfuzz import process

keywords = ['iPhone', 'Galaxy', 'Pixel']
text = "He purchased an ipone and galaxi."
tokens = text.split()

matched = [
    process.extractOne(token, keywords, score_cutoff=60)
    for token in tokens
]

print(matched)

[('iPhone', 60.00000000000001, 0), None, None, ('iPhone', 72.72727272727273, 0), None, ('Galaxy', 61.53846153846154, 1)]


In [37]:
from flashtext import KeywordProcessor

keyword_processor = KeywordProcessor()

# Dictionary: {alias: standard form}
keyword_dict = {
    'iPhone 14': ['iPhone', 'pixel'],
    'iPhone 15': ['iPhone15'],
    'Galaxy S22': ['Galaxy'],
    'Galaxy S23': ['Galaxies']
}

keyword_processor.add_keywords_from_dict(keyword_dict)

text = "He bought an iPhone 14 and pixel and she prefers the Galaxies S23."

print(keyword_processor.extract_keywords(text))


['iPhone 14', 'iPhone 14', 'Galaxy S23']


In [33]:
!pip install flashtext

Using legacy 'setup.py install' for flashtext, since package 'wheel' is not installed.
    Running setup.py install for flashtext ... done
You should consider upgrading via the '/Users/abhishekbairagi/Desktop/exp/envs/base/bin/python3 -m pip install --upgrade pip' command.
